### Imports

In [7]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from siml import MultiClassClassificationSimulation
import pandas as pd
import numpy as np

### Parameter definitions

In [8]:
# Set numpy seed
np.random.seed(42)

# Parameters for data generation
n_samples = 1000
n_features = 20
n_classes = 3
n_informative = 5
n_redundant = 2
test_training_split = 0.02

# Parameters for the scheduling
n_jobs = int(n_samples * test_training_split)
jobs = list(range(n_jobs))
weights = np.random.randint(10, 100, size=n_classes)
durations = np.random.randint(1, 10, size=n_jobs)

### Functions related to the scheduling problem

In [9]:
# Solve the scheduling problem using the STP rule
def solve_scheduling(job_class):
    job_weight = [weights[job_class[j]] for j in jobs]
    ratios = [durations[j] / job_weight[j] for j in jobs]
    solution = [x for _, x in sorted(zip(ratios, jobs), reverse=True)]
    return solution

# Compute the weighted completion time of a solution
def compute_weighted_completion_time(job_class, solution):
    job_weight = [weights[job_class[j]] for j in jobs]
    completion_time = 0
    weighted_completion_time = 0
    for j in solution:
        completion_time += durations[j]
        weighted_completion_time += job_weight[j] * completion_time
    return weighted_completion_time


### Functions related to the confusion matrices

In [10]:
# Compute (macro) TPR and FPR from confusion matrix
def compute_TPR_FPR(conf_matrix, all_classes):
    TPR_m = {cls: 0 for cls in all_classes}
    FPR_m = {cls: 0 for cls in all_classes}
    for i, cls in enumerate(all_classes):            
        TP = conf_matrix[i, i]
        FN = conf_matrix[i, :].sum() - TP
        FP = conf_matrix[:, i].sum() - TP
        TN = conf_matrix.sum() - (TP + FN + FP)            
        TPR_m[cls] = (TP / (TP + FN) if (TP + FN) > 0 else 0)
        FPR_m[cls] = (FP / (FP + TN) if (FP + TN) > 0 else 0)
        
    macro_TPR_m = np.mean(list(TPR_m.values()))
    macro_FPR_m = np.mean(list(FPR_m.values()))

    if len(all_classes) == 2:
        macro_TPR_m = list(TPR_m.values())[0]
        macro_FPR_m = list(FPR_m.values())[0]
    
    return TPR_m, FPR_m, macro_TPR_m, macro_FPR_m

# Generate all sum of n with n non-negative integers
def generate_ordered_partitions(n, m):
    result = []
    def backtrack(path, remaining, depth):
        if depth == m:
            if remaining == 0:
                result.append(path[:])
            return
        for i in range(remaining + 1):  # allow zero
            path.append(i)
            backtrack(path, remaining - i, depth + 1)
            path.pop()

    backtrack([], n, 0)
    return result

# Generate all valid confusion matrices for a given number of classes and samples
def generate_valid_confusion_matrices(n_classes, n_per_class):
    possible_row_values = []
    for c in range(n_classes):
        possible_row_values.append(generate_ordered_partitions(n_per_class[c], n_classes))

    matrices = []
    for row_0 in possible_row_values[0]:
        for row_1 in possible_row_values[1]:
            for row_2 in possible_row_values[2]:                 
                matrices.append(np.array([row_0, row_1, row_2]))

    return matrices


### Generate synthetic dataset and get the true optimal objective value

In [11]:
# Generate a multiclass classification dataset
X, y = make_classification(n_samples=n_samples,
                           n_features=n_features,
                           n_classes=n_classes,
                           n_informative=n_informative,
                           n_redundant=n_redundant,
                           class_sep=0.5,  # less separation between classes
                           flip_y=0.2,  # label noise
                           random_state=42)

X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(n_features)])
y = pd.Series(y, name="class_label")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_training_split, random_state=42)
y_test = y_test.reset_index(drop=True)
all_classes = np.unique(y)
n_per_class = ()
for cls in all_classes:
    n_per_class += (y_test.value_counts()[cls],)

# Solve the optimization problem using y_test
true_solution = solve_scheduling(y_test)
true_optimal_makespan = compute_weighted_completion_time(y_test, true_solution)

### Generate all valid confusion matrices and simulate their predictions

In [ ]:
# Try to simulate all possible confusion n_classes x n_classes matrices for the y_test instances
matrices = generate_valid_confusion_matrices(n_classes, n_per_class)

for m, matrix in enumerate(matrices):

    # Calculate macro TPR and FPR for the generated matrix
    TPR_m, FPR_m, macro_TPR_m, macro_FPR_m = compute_TPR_FPR(matrix, all_classes)

    # Simulate predictions using the generated TPR and FPR
    macro_TPR_sims = []
    macro_FPR_sims = []
    gaps = []
    obj_vals = []
    for seed in range(10):
        siml = MultiClassClassificationSimulation(seed=seed)
        y_simulated = [siml.simulate(tc, TPR_m, FPR_m) for tc in y_test]
        conf_matrix_sim = confusion_matrix(y_test, y_simulated, labels=all_classes)
        TPR_sim, FPR_sim, macro_TPR_sim, macro_FPR_sim = compute_TPR_FPR(conf_matrix_sim, all_classes)

        macro_TPR_sims.append(macro_TPR_sim)
        macro_FPR_sims.append(macro_FPR_sim)

        # Solve the optimization problem using y_simulated
        sim_solution = solve_scheduling(y_simulated)
        sim_makespan = compute_weighted_completion_time(y_test, sim_solution)
        gap = (true_optimal_makespan - sim_makespan) / true_optimal_makespan
        gaps.append(gap)
        obj_vals.append(sim_makespan)

    print(f"{m};{macro_TPR_m};{np.mean(macro_TPR_sims)};{np.std(macro_TPR_sims)};{np.min(macro_TPR_sims)};{np.max(macro_TPR_sims)};{macro_FPR_m};{np.mean(macro_FPR_sims)};{np.std(macro_FPR_sims)};{np.min(macro_FPR_sims)};{np.max(macro_FPR_sims)};{np.mean(obj_vals)};{np.std(obj_vals)};{np.mean(gaps)};{np.std(gaps)}")

### Train actual models on the dataset

In [ ]:
# Train XGB and solve
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
conf_matrix_xgb = confusion_matrix(y_test, y_xgb, labels=all_classes)
TPR_xgb, FPR_xgb, macro_TPR_xgb, macro_FPR_xgb = compute_TPR_FPR(conf_matrix_xgb, all_classes)
xgb_solution = solve_scheduling(y_xgb)
xgb_makespan = compute_weighted_completion_time(y_test, xgb_solution)
xgb_gap = (true_optimal_makespan - xgb_makespan) / true_optimal_makespan
print(f"XGB;{macro_TPR_xgb};{macro_FPR_xgb};{xgb_makespan};{xgb_gap}")

# Train LR and solve
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_lr = lr.predict(X_test)
conf_matrix_lr = confusion_matrix(y_test, y_lr, labels=all_classes)
TPR_lr, FPR_lr, macro_TPR_lr, macro_FPR_lr = compute_TPR_FPR(conf_matrix_lr, all_classes)
lr_solution = solve_scheduling(y_lr)
lr_makespan = compute_weighted_completion_time(y_test, lr_solution)
lr_gap = (true_optimal_makespan - lr_makespan) / true_optimal_makespan
print(f"LR;{macro_TPR_lr};{macro_FPR_lr};{lr_makespan};{lr_gap}")

# Train RF and solve
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
conf_matrix_rf = confusion_matrix(y_test, y_rf, labels=all_classes)
TPR_rf, FPR_rf, macro_TPR_rf, macro_FPR_rf = compute_TPR_FPR(conf_matrix_rf, all_classes)
rf_solution = solve_scheduling(y_rf)
rf_makespan = compute_weighted_completion_time(y_test, rf_solution)
rf_gap = (true_optimal_makespan - rf_makespan) / true_optimal_makespan
print(f"RF;{macro_TPR_rf};{macro_FPR_rf};{rf_makespan};{rf_gap}")